# Importing Libraries

* System Append to set proper path

In [ ]:
sys.path.append('../')

* Default

In [ ]:
import lasio
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
import time
import multiprocessing

* Pandas Options

In [ ]:
pd.set_option('max_columns', None)

* Source Code

In [ ]:
from Source.Utils import welllog
from Source.Utils import multi_df
from Source.Utils import well_plot

* Tqdm Progress Bar

In [ ]:
%%capture
from tqdm import tqdm_notebook

# Reading Miscellaneous Data

In [ ]:
file_path = '../Data/Lithology code data.xlsx'

file_path2 = '../Data/NPD stratigraphic picks north sea.xlsx'

litho_code = pd.read_excel(file_path)

picks = pd.read_excel(file_path2)

* Lithology Code 

In [ ]:
litho_code.head(n=2)

        * Fixing Columns IDs

In [ ]:
litho_code.columns = litho_code.iloc[0]

        * Droping first row (as it has become the columns ids now)

In [ ]:
litho_code.drop(index=0, inplace=True)

        * Checking Final Result

In [ ]:
litho_code.head(n=2)

* Stratigraphic Picks

In [ ]:
picks.head(n=2)

        * Renaming Unidentified Column

In [ ]:
picks.rename(columns={'Unnamed: 0': 'Unidentified Column'}, inplace=True)

        * Checking Result

In [ ]:
picks.head(n=2)

# Reading Las files Dataset

In [ ]:
path = '../Data/GEOLINK_Lithology and wells NORTH SEA/'

npd_wells = welllog.read_las_directory(path)

* Las files Glance

In [ ]:
print('Number of Las Files read: ' + str(len(npd_wells)))
print('##########################')
print('Las files ID: ' + str(npd_wells.keys()))
print('##########################')
print(str(npd_wells['15_9-12'].curves))


In [ ]:
npd_wells['15_9-12'].header

        Note: The Mnmonic Table above does not necessarily represent all the available log curves on the dataset

# Main Dataframe Building and Processing

* Checking unmatching unit of measurement for each log curve

In [ ]:
unit_mismatch_list = welllog.unit_check(npd_wells)

* Converting all las files to dataframe

In [ ]:
npd_wells_df = {}

for id in tqdm_notebook(list(npd_wells.keys()), desc='Converting to dataframe'):

    npd_wells_df[id] = npd_wells[id].df()

        * Filling in Log Dataframes

In [ ]:
#logs_dict = welllog.log_frame(npd_wells_df, logs_list, mode='df')

* Creating Main Dataframe

        * Creating Well ID column

In [ ]:
for id in tqdm_notebook(list(npd_wells_df.keys()), desc='Adding Well Name Column'):

    npd_wells_df[id]['WELL_NAME'] = id

        * Converting Depth to a Column

In [ ]:
for id in tqdm_notebook(list(npd_wells_df.keys()), desc='Adding Depth Column'):

    npd_wells_df[id]['DEPTH'] = npd_wells_df[id].index

        * Selected Logs

In [ ]:
logs_list = ['DEPTH','LITHOLOGY_GEOLINK', 'CALI', 'NPHI', 'RHOB', 'GR', 'DTC', 'RDEP', 'WELL_NAME']


        * Creating Empty Dataframe

In [ ]:
df_main = pd.DataFrame(columns= logs_list)

        * Filling Dataframe

In [ ]:
for id in tqdm_notebook(list(npd_wells_df.keys()), desc='Adding Depth Column'):

    tmp = []

    for i in range(len(logs_list)):

        if logs_list[i] in npd_wells_df[id].columns:

            tmp.append(logs_list[i])

    df_main = df_main.append(npd_wells_df[id][tmp], ignore_index=True)     

* Lithology Code Dictionary

In [ ]:
litho_code_dict = {}

for row_val in litho_code.index:

    litho_code_dict[litho_code['Abbreviation'][row_val]] = litho_code['Lithology Attribute'][row_val]

In [ ]:
litho_code_dict

## Exploratory Data Analysis

* Correlation Matrix

In [ ]:
corrmat_df = abs(df_main.corr()) # absolute correlation

plt.figure(figsize=(15,10))

sns.heatmap(corrmat_df, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, cmap='Blues')

* Mixer Scatter Plot

In [ ]:
plt.figure(figsize=(15,10))

sns.set()

sns.pairplot(df_main.dropna().sample(1000))
plt.show();

* General Statistics

In [ ]:
df_main.describe()

* Distribution Information

        * Lithology_Geoling Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('LITHOLOGY_GEOLINK distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * CALI Variable 

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.CALI.notnull()].CALI.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.CALI.notnull()].CALI.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('CALI distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * NPHI Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.NPHI.notnull()].NPHI.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.NPHI.notnull()].NPHI.values);
#plt.xlim(0,100) 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('NPHI distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

In [ ]:
multiprocessing.set_start_method('spawn', True)

start_time = time.time()

NUM_CORES = 30

df_chunks_out_count = np.array_split(df_main, NUM_CORES)

with multiprocessing.Pool(NUM_CORES) as pool:

    df_main = pd.concat(pool.map(multi_df.nphi_filtering, df_chunks_out_count), ignore_index=True)

print(str(round((time.time() - start_time)/60,1)) + ' minutes taken') 

        * RHOB Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.RHOB.notnull()].RHOB.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.RHOB.notnull()].RHOB.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('RHOB distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * GR Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.GR.notnull()].GR.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.GR.notnull()].GR.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('GR distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * DTC Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.DTC.notnull()].DTC.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.DTC.notnull()].DTC.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('DTC distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * RDEP Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.RDEP.notnull()].RDEP.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.RDEP.notnull()].RDEP.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('RDEP distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

* Relationship with the categorical variable

        * CALI

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].CALI], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='CALI', data=data)


        * NPHI

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].NPHI], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='NPHI', data=data)


        * RHOB

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].RHOB], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='RHOB', data=data)


        * GR

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].GR], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='GR', data=data)


        * DTC

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].DTC], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='DTC', data=data)


        * RDEP

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].RDEP], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='RDEP', data=data)


* Outliers Classification

        * Number of outliers per row column

In [ ]:
val = 'Number of Outliers'

df_main[val] = 0

        * Non-Outlier Ranges (Expert Provided) (Used inside classify_outliers function! In here only for display purpose)

In [ ]:
ranges = {}

ranges['CALI'] = [0, 30]

ranges['NPHI'] = [0.1, 0.65] 

ranges['RHOB'] = [1, 4] 

ranges['GR'] = [0, 200]

ranges['DTC'] = [40, 200]

ranges['RDEP'] = [0.0001, 6000] 

        * Filling in the Numbers of Outliers Column

In [ ]:
multiprocessing.set_start_method('spawn', True)

start_time = time.time()

NUM_CORES = 30

df_chunks_out_count = np.array_split(df_main, NUM_CORES)

with multiprocessing.Pool(NUM_CORES) as pool:

    df_main = pd.concat(pool.map(multi_df.classify_outliers, df_chunks_out_count), ignore_index=True)

print(str(round((time.time() - start_time)/60,1)) + ' minutes taken') 

        * Dropping Outliers

In [ ]:
start_time = time.time()

NUM_CORES = 30

df_chunks_out_count = np.array_split(df_main, NUM_CORES)

with multiprocessing.Pool(NUM_CORES) as pool:

    df_main = pd.concat(pool.map(multi_df.remove_outliers_class, df_chunks_out_count), ignore_index=True)

print(str(round((time.time() - start_time)/60,1)) + ' minutes taken') 

* Null Values Analysis

        * Raw Count

In [ ]:
columns = df_main.columns.to_list()

columns.remove('LITHOLOGY_GEOLINK')

columns.remove('DEPTH')

columns.remove('WELL_NAME')

columns.remove('Number of Outliers')

In [ ]:
plt.figure(figsize=(15,10))

sns.barplot(x=df_main.isnull().sum().index, y=df_main.isnull().sum()) # only considering variables of interest
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Total missing values (Not a Number)', fontsize=10) # Not counting -999,25
plt.title('Total missing values (Not a Number)', fontsize=15) # Not counting -999,25

        * Closer Look

In [ ]:
plt.figure(figsize=(15,10))

sns.barplot(x=df_main[columns].isnull().sum().index, y=df_main[columns].isnull().sum()) # only considering variables of interest
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Total missing values (Not a Number)', fontsize=10) # Not counting -999,25
plt.title('Total missing values (Not a Number)', fontsize=15) # Not counting -999,25

        * Dropping Rows for Null values in Pivot Columns

In [ ]:
df_main = df_main.dropna(subset=['CALI', 'RHOB', 'GR', 'DTC', 'RDEP'])

In [ ]:
plt.figure(figsize=(15,10))

sns.barplot(x=df_main.isnull().sum().index, y=df_main.isnull().sum()) # only considering variables of interest
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Total missing values (Not a Number)', fontsize=10) 
plt.title('Total missing values (Not a Number)', fontsize=15) 

# Checkpoint

In [ ]:
path_file_lito_code = '../checkpoints/litho_code.csv.gz'

path_file_picks = '../checkpoints/picks.csv.gz'

path_file_df_main = '../checkpoints/df_main.csv.gz'

litho_code.to_csv(path_file_lito_code,index=False, compression='gzip')

picks.to_csv(path_file_picks,index=False, compression='gzip')

df_main.to_csv(path_file_df_main,index=False, compression='gzip')